In [1]:
import sys
import os
import pandas as pd

from bokeh.palettes import Category10


import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


#Adjust the working folder
file_folder = globals()['_dh'][0]
wk_dir = os.path.dirname(file_folder)
os.chdir(wk_dir)
from core.feature import *
from core.estimate import *
from core.db import *
# from core.predict import *
# from core.merge import *
# from core.merge import *

from file_cache.utils.util_pandas import *
from file_cache.cache import file_cache
from file_cache.utils.util_log import *

logging.getLogger().setLevel(logging.INFO)

import argparse
sys.argv = ['program',  '-W' , '--gp_name', 'lr_bin_9', '--shift', 0]



logging.getLogger().setLevel(logging.INFO)
!hostname

['week_day', 'stationID', 'time_ex', 'bin_id', 'p', 'd', 'q', 'out_1', 'in_1', 'out_1_p', 'in_1_p', 'out_2', 'in_2', 'out_2_p', 'in_2_p', 'out_3', 'in_3', 'out_3_p', 'in_3_p']


2019-03-22 17:16:33,648 140296147416896 util_log.py[93] INFO Start the program at:ai-prd-06, 173.36.99.84, with:Load module


yes
ai-prd-06


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import norm, rankdata
import warnings
import gc
import os
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)

In [3]:
path = 'input'

test_28 = path + '/Metro_testA/testA_record_2019-01-28.csv'

In [4]:
@file_cache()
def get_base_features(file_path):
    df = pd.read_csv(file_path)
    
    #df = df_.copy()
    
    # base time
    df['day']     = (pd.to_datetime(df['time'])-pd.to_datetime('2019-01-01')).dt.days+1
    df['week']    = pd.to_datetime(df['time']).dt.dayofweek + 1
    df['weekend'] = (pd.to_datetime(df.time).dt.weekday >=5).astype(int)
    df['hour']    = df['time'].apply(lambda x: int(x[11:13]))
    df['minute']  = df['time'].apply(lambda x: int(x[14:15]+'0'))
    
    # count,sum
    result = df.groupby(['stationID', 'week', 'weekend', 'day', 'hour', 'minute']).status.agg(['count', 'sum']).reset_index()
    
    # nunique
    tmp     = df.groupby(['stationID'])['deviceID'].nunique().reset_index(name='nuni_deviceID_of_stationID')
    result  = result.merge(tmp, on=['stationID'], how='left')
    tmp     = df.groupby(['stationID','hour'])['deviceID'].nunique().reset_index(name='nuni_deviceID_of_stationID_hour')
    result  = result.merge(tmp, on=['stationID','hour'], how='left')
    tmp     = df.groupby(['stationID','hour','minute'])['deviceID'].nunique().\
                                           reset_index(name='nuni_deviceID_of_stationID_hour_minute')
    result  = result.merge(tmp, on=['stationID','hour','minute'], how='left')
    
    # in,out
    result['inNums']  = result['sum']
    result['outNums'] = result['count'] - result['sum']
    
    #
    result['day_since_first'] = result['day'] - 1 
    result.fillna(0, inplace=True)
    del result['sum'],result['count']
    
    return result

data = get_base_features(test_28)
data.head()

2019-03-22 17:16:35,007 140296147416896 util_log.py[48] INFO get_base_features begin with(1 paras) :['input/Metro_testA/testA_record_2019-01-28.csv'], []
2019-03-22 17:16:35,320 140296147416896 util_log.py[63] INFO cost 00.3 sec:get_base_features(['input/Metro_testA/testA_record_2019-01-28.csv'], []), return:DF:(9124, 12), end 


,stationID,week,weekend,day,hour,minute,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,inNums,outNums,day_since_first
0,0,1,0,28,0,0,18,3,1,0,1,27
1,0,1,0,28,0,10,18,3,2,0,2,27
2,0,1,0,28,4,30,18,1,1,0,1,27
3,0,1,0,28,5,30,18,11,5,22,0,27
4,0,1,0,28,5,40,18,11,7,14,0,27


In [5]:
# data_list = os.listdir(path+'/Metro_train/')
# for i in range(0, len(data_list)):
#     if data_list[i].split('.')[-1] == 'csv':
#         print(data_list[i], i)       
#         df = get_base_features(path+'/Metro_train/' + data_list[i])
#         data = pd.concat([data, df], axis=0, ignore_index=True)
#     else:
#         continue
        
@file_cache()
def get_data():
    data = get_base_features(test_28)
    data_list = os.listdir(path+'/Metro_train/')
    for i in range(0, len(data_list)):
        if data_list[i].split('.')[-1] == 'csv':
            print(data_list[i], i)
            df = pd.read_csv(path+'/Metro_train/' + data_list[i])
            df = get_base_features(df)
            data = pd.concat([data, df], axis=0, ignore_index=True)
        else:
            continue
    return data

data = get_data()

2019-03-22 17:16:35,339 140296147416896 util_log.py[48] INFO get_data begin with(0 paras) :[], []
2019-03-22 17:16:35,448 140296147416896 util_log.py[63] INFO cost 00.1 sec:get_data([], []), return:DF:(234847, 12), end 


In [6]:
# 剔除周末,并修改为连续时间
data = data[(data.day!=5)&(data.day!=6)]
data = data[(data.day!=12)&(data.day!=13)]
data = data[(data.day!=19)&(data.day!=20)]
data = data[(data.day!=26)&(data.day!=27)]

# def fix_day(d):
#     if d in [1,2,3,4]:
#         return d
#     elif d in [7,8,9,10,11]:
#         return d - 2
#     elif d in [14,15,16,17,18]:
#         return d - 4
#     elif d in [21,22,23,24,25]:
#         return d - 6
#     elif d in [28]:
#         return d - 8
# data['day'] = data['day'].apply(fix_day)

In [7]:
tmp = data.copy()
tmp[tmp.day==1].head()

,stationID,week,weekend,day,hour,minute,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,inNums,outNums,day_since_first
18097,0,2,0,1,5,30,18,5,1,1,0,0
18098,0,2,0,1,5,40,18,5,2,4,0,0
18099,0,2,0,1,5,50,18,5,4,3,1,0
18100,0,2,0,1,6,0,18,18,5,17,0,0
18101,0,2,0,1,6,10,18,18,6,21,0,0


In [8]:
test = pd.read_csv(path + '/Metro_testA/testA_submit_2019-01-29.csv')
test['week']    = pd.to_datetime(test['startTime']).dt.dayofweek + 1
test['weekend'] = (pd.to_datetime(test.startTime).dt.weekday >=5).astype(int)
test['day']     = test['startTime'].apply(lambda x: int(x[8:10]))
test['hour']    = test['startTime'].apply(lambda x: int(x[11:13]))
test['minute']  = test['startTime'].apply(lambda x: int(x[14:15]+'0'))
test['day_since_first'] = test['day'] - 1
test = test.drop(['startTime','endTime'], axis=1)
data = pd.concat([data,test], axis=0, ignore_index=True)

stat_columns = ['inNums','outNums']

In [35]:
def get_refer_day(d):
    sn = [item for item in range(40) if item%7 not in [5,6] ]
    sn_map = dict(zip(sn[:-1], sn[1:]))
    return sn_map[d]
print(get_refer_day(28))

29


In [31]:

tmp = data.copy()
# tmp_df = tmp[tmp.day==1]
# tmp_df['day'] = tmp_df['day'] - 1
# print(tmp_df.shape)
# tmp = pd.concat([tmp, tmp_df], axis=0, ignore_index=True)
# print(tmp.shape)
tmp['day'] = tmp['day'].apply(get_refer_day)
#tmp.day.value_counts().sort_index(), data.day.value_counts().sort_index()



In [33]:
 


for f in stat_columns:
    tmp.rename(columns={f: f+'_last'}, inplace=True) 
    
tmp = tmp[['stationID','day','hour','minute','inNums_last','outNums_last']]

print(data.shape, tmp.shape)
data = data.merge(tmp, on=['stationID','day','hour','minute'], how='left')
data.fillna(0, inplace=True)
data[data.day==29]

(120531, 28) (120531, 8)


,day,day_since_first,hour,inNums,minute,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,outNums,stationID,week,weekend,inNums_last_x,outNums_last_x,inNums_whm_max,inNums_whm_min,inNums_whm_mean,outNums_whm_max,outNums_whm_min,outNums_whm_mean,inNums_wh_max,inNums_wh_min,inNums_wh_mean,outNums_wh_mean,inNums_last_y,inNums_last_y,outNums_last_y,outNums_last_y,inNums_last,inNums_last,outNums_last,outNums_last
108867,29,28,0,0.0,0,0.0,0.0,0.0,0.0,0,2,0,0.0,1.0,0.0,0.0,0.000000,3.0,0.0,1.500000,0.0,0.0,0.000000,0.428571,0.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0
108868,29,28,0,0.0,10,0.0,0.0,0.0,0.0,0,2,0,0.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.428571,0.0,0.0,2.0,3.0,0.0,0.0,2.0,3.0
108869,29,28,0,0.0,20,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108870,29,28,0,0.0,30,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108871,29,28,0,0.0,40,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108872,29,28,0,0.0,50,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108873,29,28,1,0.0,0,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108874,29,28,1,0.0,10,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108875,29,28,1,0.0,20,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108876,29,28,1,0.0,30,0.0,0.0,0.0,0.0,0,2,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
tmp = data.groupby(['stationID','week','hour','minute'], as_index=False)['inNums'].agg({
                                                                        'inNums_whm_max'    : 'max',
                                                                        'inNums_whm_min'    : 'min',
                                                                        'inNums_whm_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour','minute'], how='left')

tmp = data.groupby(['stationID','week','hour','minute'], as_index=False)['outNums'].agg({
                                                                        'outNums_whm_max'    : 'max',
                                                                        'outNums_whm_min'    : 'min',
                                                                        'outNums_whm_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour','minute'], how='left')

tmp = data.groupby(['stationID','week','hour'], as_index=False)['inNums'].agg({
                                                                        'inNums_wh_max'    : 'max',
                                                                        'inNums_wh_min'    : 'min',
                                                                        'inNums_wh_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour'], how='left')

tmp = data.groupby(['stationID','week','hour'], as_index=False)['outNums'].agg({
                                                                        #'outNums_wh_max'    : 'max',
                                                                        #'outNums_wh_min'    : 'min',
                                                                        'outNums_wh_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour'], how='left')

In [ ]:
#data=data[data.day>1]

In [21]:
# def recover_day(d):
#     if d in [1,2,3,4]:
#         return d
#     elif d in [5,6,7,8,9]:
#         return d + 2
#     elif d in [10,11,12,13,14]:
#         return d + 4
#     elif d in [15,16,17,18,19]:
#         return d + 6
#     elif d == 20:
#         return d + 8
#     else:
#         return d



last_day = 28
predict_day = last_day + 1

all_columns = [f for f in data.columns if f not in ['weekend','inNums','outNums']]
### all data
all_data = data[data.day!=predict_day]
#all_data['day'] = all_data['day'].apply(recover_day)
X_data = all_data[all_columns].values

train = data[data.day <last_day]
#train['day'] = train['day'].apply(recover_day)
X_train = train[all_columns].values

valid = data[data.day==last_day]
#valid['day'] = valid['day'].apply(recover_day)
X_valid = valid[all_columns].values

test  = data[data.day==predict_day]
X_test = test[all_columns].values

In [22]:
!hostname

ai-prd-06


In [23]:
print(all_data.shape, train.shape, valid.shape)
all_data.head()
#all_data.to_csv('./cache/output.csv',index=None)

(108867, 24) (99743, 24) (9124, 24)


,day,day_since_first,hour,inNums,minute,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,outNums,stationID,week,weekend,inNums_last,outNums_last,inNums_whm_max,inNums_whm_min,inNums_whm_mean,outNums_whm_max,outNums_whm_min,outNums_whm_mean,inNums_wh_max,inNums_wh_min,inNums_wh_mean,outNums_wh_mean
0,28,27,0,0.0,0,18.0,3.0,1.0,1.0,0,1,0,0.0,2.0,0.0,0.0,0.000000,1.0,1.0,1.000000,2.0,0.0,0.666667,1.000000
1,28,27,0,0.0,10,18.0,3.0,2.0,2.0,0,1,0,0.0,3.0,0.0,0.0,0.000000,2.0,2.0,2.000000,2.0,0.0,0.666667,1.000000
2,28,27,4,0.0,30,18.0,1.0,1.0,1.0,0,1,0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,1.000000,0.0,0.0,0.000000,1.000000
3,28,27,5,22.0,30,18.0,11.0,5.0,0.0,0,1,0,21.0,0.0,22.0,3.0,9.666667,1.0,0.0,0.333333,53.0,3.0,22.454545,0.818182
4,28,27,5,14.0,40,18.0,11.0,7.0,0.0,0,1,0,22.0,0.0,30.0,14.0,20.000000,1.0,0.0,0.250000,53.0,3.0,22.454545,0.818182


In [24]:
all_data.day.value_counts().sort_index()

11    9006
14    8946
15    9023
16    8974
17    9072
18    8984
21    9069
22    9153
23    9188
24    9147
25    9181
28    9124
Name: day, dtype: int64

In [25]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2
}

######################################################inNums
y_train = train['inNums']
y_valid = valid['inNums']
y_data  = all_data['inNums']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_evals = lgb.Dataset(X_valid, y_valid , reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                early_stopping_rounds=200,
                verbose_eval=1000,
                )
in_score = gbm.best_score['valid']['l1']

### all_data
lgb_train = lgb.Dataset(X_data, y_data)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=gbm.best_iteration,
                valid_sets=[lgb_train],
                valid_names=['train'],
                verbose_eval=1000,
                )
test['inNums'] = gbm.predict(X_test)

######################################################outNums
y_train = train['outNums']
y_valid = valid['outNums']
y_data  = all_data['outNums']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_evals = lgb.Dataset(X_valid, y_valid , reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                early_stopping_rounds=200,
                verbose_eval=1000,
                )
out_score = gbm.best_score['valid']['l1']

### all_data
lgb_train = lgb.Dataset(X_data, y_data)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=gbm.best_iteration,
                valid_sets=[lgb_train],
                valid_names=['train'],
                verbose_eval=1000,
                )
test['outNums'] = gbm.predict(X_test)

sub = pd.read_csv(path + '/Metro_testA/testA_submit_2019-01-29.csv')
sub['inNums']   = test['inNums'].values
sub['outNums']  = test['outNums'].values
# 结果修正
sub.loc[sub.inNums<0 , 'inNums']  = 0
sub.loc[sub.outNums<0, 'outNums'] = 0



Training until validation scores don't improve for 200 rounds.
[1000]	train's l1: 10.4887	valid's l1: 13.3202
[2000]	train's l1: 9.80157	valid's l1: 13.2013
Early stopping, best iteration is:
[2367]	train's l1: 9.60975	valid's l1: 13.173
[1000]	train's l1: 10.595
[2000]	train's l1: 9.92283
Training until validation scores don't improve for 200 rounds.
[1000]	train's l1: 12.0517	valid's l1: 15.5561
[2000]	train's l1: 11.3062	valid's l1: 15.4065
[3000]	train's l1: 10.7537	valid's l1: 15.3218
[4000]	train's l1: 10.2904	valid's l1: 15.2565
[5000]	train's l1: 9.88802	valid's l1: 15.2257
Early stopping, best iteration is:
[5290]	train's l1: 9.78096	valid's l1: 15.2148
[1000]	train's l1: 12.0969
[2000]	train's l1: 11.3507
[3000]	train's l1: 10.8012
[4000]	train's l1: 10.351
[5000]	train's l1: 9.96663


In [26]:
avg = (in_score+out_score)/2
file_name = f'output/sub_{avg:06.4f}_{in_score:06.4f}_{out_score:06.4f}_{int(time.time() % 10000000)}.csv'
logger.info(file_name)

#13.2464/15.0891
#13.1399/15.054/14.0970
#14.0970_13.1399_15.0540


2019-03-22 17:34:16,755 140296147416896 <ipython-input-26-ca7a8c0da8f0>[3] INFO output/sub_14.1939_13.1730_15.2148_3301256.csv


In [27]:
sub[['stationID', 'startTime', 'endTime', 'inNums', 'outNums']].to_csv(file_name, index=False)